# Comparing spatial pattern of velocity response to forcing

In this notebook, we'll use the `iceutils` package (Bryan Riel) to invert continuous time-varying surface velocity fields on Helheim Glacier.  We'll then process several observational datasets (gathered by Denis Felikson) using the `nifl` module (Lizz Ultee) and compare time series of these variables against surface velocity at several points.  Finally, we'll visualize spatial differences in the relationship between surface velocity and each hypothesised forcing variable.

Last updated: 6 Jan 2022 by Lizz Ultee

### Import the necessary packages

In [ ]:
from netCDF4 import Dataset
from scipy import interpolate
import pyproj as pyproj
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from mpl_toolkits.axes_grid1 import make_axes_locatable 
import iceutils as ice
import nifl_helper as nifl

### Define where the necessary data lives

In [ ]:
flowline_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Felikson-flowlines/netcdfs/glaciera199.nc'
velocity_fpath='/Users/lizz/Documents/GitHub/Data_unsynced/Gld-Stack/'
gl_bed_fpath ='/Users/lizz/Documents/GitHub/Data_unsynced/BedMachine-Greenland/BedMachineGreenland-2017-09-20.nc'
catchment_smb_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Helheim-processed/smb_rec._.BN_RACMO2.3p2_ERA5_3h_FGRN055.1km.MM.csv'
runoff_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Helheim-processed/runoff._.BN_RACMO2.3p2_ERA5_3h_FGRN055.1km.MM.csv'
termini_fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/Helheim-processed/HLM_terminus_widthAVE.csv'

### Define the domain of analysis

We will analyse along flowlines defined by Denis Felikson in his previous work, saved and shared as NetCDF files.  The flowlines are numbered 01-10 across the terminus; flowline 05 is close to the middle.  Note that Helheim Glacier has two large branches.  For now we'll study the main trunk, `glaciera199.nc`.  The more southerly trunk is `glacierb199.nc`.

In [ ]:
ncfile = Dataset(flowline_fpath, 'r')
xh = ncfile['flowline05'].variables['x'][:]
yh = ncfile['flowline05'].variables['y'][:]
ncfile.close()

In [ ]:
## Define points at which to extract
upstream_max = 500 # index of last xh,yh within given distance of terminus--pts roughly 50m apart
xys = [(xh[i], yh[i]) for i in range(0, upstream_max, 20)][2::]

## Import and invert velocity observations

In [ ]:
## Set up combined hdf5 stack
hel_stack = ice.MagStack(files=[velocity_fpath+'vx.h5', velocity_fpath+'vy.h5'])
data_key = 'igram' # B. Riel convention for access to datasets in hdf5 stack

In [ ]:
# Create an evenly spaced time array for time series predictions
t_grid = np.linspace(hel_stack.tdec[0], hel_stack.tdec[-1], 1000)

# First convert the time vectors to a list of datetime
dates = ice.tdec2datestr(hel_stack.tdec, returndate=True)
dates_grid = ice.tdec2datestr(t_grid, returndate=True)

# Build the collection
collection = nifl.build_collection(dates)

# Construct a priori covariance
Cm = nifl.computeCm(collection)
iCm = np.linalg.inv(Cm)

# Instantiate a model for inversion
model = ice.tseries.Model(dates, collection=collection)

# Instantiate a model for prediction
model_pred = ice.tseries.Model(dates_grid, collection=collection)

## Access the design matrix for plotting
G = model.G

# Create lasso regression solver that does the following:
# i) Uses an a priori covariance matrix for damping out the B-splines
# ii) Uses sparsity-enforcing regularization (lasso) on the integrated B-splines
solver = ice.tseries.select_solver('lasso', reg_indices=model.itransient, penalty=0.05,
                                   rw_iter=1, regMat=iCm)

Now that we are set up with our data and machinery, we'll ask the inversion to make us a continuous time series of velocity at each point we wish to study.

In [ ]:
preds = []
for j, xy in enumerate(xys):
    try:
        pred, st, lt = nifl.VSeriesAtPoint(xy, vel_stack=hel_stack, collection=collection, 
                                  model=model, model_pred=model_pred, solver=solver, 
                                  t_grid=t_grid, sigma=2.5, data_key='igram')
        preds.append(pred)
    except AssertionError: # catches failed inversion
        print('Insufficient data for point {}. Removing'.format(j))
        xys.remove(xy)
        continue

Test to confirm that `xys` has been trimmed to only those points for which a valid prediction can be generated -- the below should return `True`.

In [ ]:
len(xys)==len(preds)

## Comparison data sets

### Bed topography

Mostly we will use this for plotting and for defining a standard coordinate system.  However, future analyses could combine bed topography with calving position or other variables to analyse effect on surface velocity.

In [ ]:
## Read in and interpolate BedMachine topography
fh = Dataset(gl_bed_fpath, mode='r')
xx = fh.variables['x'][:].copy() #x-coord (polar stereo (70, 45))
yy = fh.variables['y'][:].copy() #y-coord
s_raw = fh.variables['surface'][:].copy() #surface elevation
h_raw=fh.variables['thickness'][:].copy() # Gridded thickness
b_raw = fh.variables['bed'][:].copy() # bed topo
thick_mask = fh.variables['mask'][:].copy()
ss = np.ma.masked_where(thick_mask !=2, s_raw)#mask values: 0=ocean, 1=ice-free land, 2=grounded ice, 3=floating ice, 4=non-Greenland land
hh = np.ma.masked_where(thick_mask !=2, h_raw) 
bb = b_raw #don't mask, to allow bed sampling from modern bathymetry (was subglacial in ~2006)
fh.close()

In [ ]:
## Interpolate in area of Helheim
xl, xr = 6100, 6600
yt, yb = 12700, 13100
x_hel = xx[xl:xr]
y_hel = yy[yt:yb]
s_hel = ss[yt:yb, xl:xr]
b_hel = bb[yt:yb, xl:xr]
S_helheim = interpolate.RectBivariateSpline(x_hel, y_hel[::-1], s_hel.T[::,::-1]) #interpolating surface elevation provided
B_helheim = interpolate.RectBivariateSpline(x_hel, y_hel[::-1], b_hel.T[::,::-1]) #interpolating bed elevation provided

### Catchment-integrated SMB

We load in a 1D timeseries of surface mass balance integrated over the whole Helheim catchment.  This data is monthly surface mass balance from the HIRHAM5 model, integrated over the Helheim catchment defined by K. Mankoff, with processing steps (coordinate reprojection, Delaunay triangulation, nearest-neighbor search and area summing) in `catchment-integrate-smb.py`.

In [ ]:
smb_racmo = pd.read_csv(catchment_smb_fpath, index_col=0, parse_dates=True)
smb_tr = smb_racmo.loc[smb_racmo.index.year >= 2006]
smb = smb_tr.loc[smb_tr.index.year <2018].squeeze() # trim dates to overlapping period

smb_d = [d.utctimetuple() for d in smb.index]
smb_d_interp = [ice.timeutils.datestr2tdec(d[0], d[1], d[2]) for d in smb_d]
smb_func = interpolate.interp1d(smb_d_interp, smb)

The time series we analyse here are autocorrelated. We must compute a correction factor to the significance limits based on the lag-1 autocorrelation, as described in Dean & Dunsmuir (2016).

In [ ]:
a_vel = sm.tsa.stattools.acf(np.diff(preds[0]['full']))[1]
b_smb = sm.tsa.stattools.acf(np.diff(smb))[1]
F_smb = np.sqrt((1+(a_vel*b_smb))/(1-(a_vel*b_smb)))

Now, we compute the normalized cross-correlation between catchment-integrated SMB and surface velocity at each point along the flowline.  We will draw on the inverted velocity series saved in `preds` above.  We save the value of the maximum normalized cross-correlation, and the value in days of the lag where it occurs, to compare with other variables later.  We also test whether each saved value is significantly different from 0, with the confidence interval around 0 modified as above.

In [ ]:
smb_corr_amax = []
smb_lag_amax = []
smb_significance = []

for xy, pred in zip(xys, preds):
    corr, lags, ci = nifl.Xcorr1D(xy, series_func=smb_func, series_dates=smb_d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=1, normalize=True, pos_only=True)
    ci_mod = F_smb*np.asarray(ci)
    smb_corr_amax.append(corr[abs(corr).argmax()])
    smb_lag_amax.append(lags[abs(corr).argmax()])
    smb_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

### Runoff

We import monthly runoff from the RACMO model, integrated over the Helheim catchment and shared as a CSV by Denis Felikson.  Because this data is catchment-integrated, we interpolate a single 1D time series that will be used at all points.

In [ ]:
runoff_racmo = pd.read_csv(runoff_fpath, index_col=0, parse_dates=True)
runoff_tr = runoff_racmo.loc[runoff_racmo.index.year >= 2006]
runoff = runoff_tr.loc[runoff_tr.index.year <2018].squeeze()

runoff_d = [d.utctimetuple() for d in runoff.index]
d_interp = [ice.timeutils.datestr2tdec(d[0], d[1], d[2]) for d in runoff_d]
runoff_func = interpolate.interp1d(d_interp, runoff)

Again, we compute a correction factor to account for autocorrelated data in the 95% confidence interval around 0.

In [ ]:
b_runoff = sm.tsa.stattools.acf(np.diff(runoff))[1]
F_runoff = np.sqrt((1+(a_vel*b_runoff))/(1-(a_vel*b_runoff)))

We compute the normalized cross-correlation between catchment-integrated runoff and surface velocity at each same point.  Again we save the value of the maximum normalized cross-correlation, and the value in days of the lag where it occurs, to compare with other variables.

In [ ]:
runoff_corr_amax = []
runoff_lag_amax = []
runoff_significance = []

for xy, pred in zip(xys, preds):
    corr, lags, ci = nifl.Xcorr1D(xy, series_func=runoff_func, series_dates=d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=1, normalize=True, pos_only=True)
    ci_mod = F_runoff*np.asarray(ci)
    runoff_corr_amax.append(corr[abs(corr).argmax()])
    runoff_lag_amax.append(lags[abs(corr).argmax()])
    runoff_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

### Terminus position change

We import width-averaged terminus position change processed by Leigh Stearns.  These data give terminus position in km from a baseline, so they do not need to be processed into a coordinate system.

In [ ]:
termini = pd.read_csv(termini_fpath, index_col=0, parse_dates=True, usecols=[0,1])
trmn = termini.loc[termini.index.year >= 2006]
tm = trmn.loc[trmn.index.year <2017].squeeze()

## smooth a little to make more comparable with SMB and runoff
td = tm.rolling('10D').mean() # approximately 3 measurements per window

termini_d = [d.utctimetuple() for d in td.index]
tm_d_interp = [ice.timeutils.datestr2tdec(d[0], d[1], d[2]) for d in termini_d]
termini_func = interpolate.interp1d(tm_d_interp, td)

And the confidence interval correction factor for these data:

In [ ]:
b_terminus = sm.tsa.stattools.acf(np.diff(td))[1]
F_terminus = np.sqrt((1+(a_vel*b_terminus))/(1-(a_vel*b_terminus)))

Finally, finding the maximum cross-correlation and the (positive) lag at which it occurs, as for other variables.

In [ ]:
terminus_corr_amax = []
terminus_lag_amax = []
terminus_significance = []

for xy, pred in zip(xys, preds):
    corr, lags, ci = nifl.Xcorr1D(xy, series_func=termini_func, series_dates=tm_d_interp, 
                              velocity_pred=pred, t_grid=t_grid, t_limits=(2009,2017), 
                              diff=1, normalize=True, pos_only=True)
    ci_mod = F_terminus *np.asarray(ci)
    terminus_corr_amax.append(corr[abs(corr).argmax()])
    terminus_lag_amax.append(lags[abs(corr).argmax()])
    terminus_significance.append(abs(corr[abs(corr).argmax()]) > ci_mod[abs(corr).argmax()])

## Spatial pattern of cross-correlation (Figure 2)

Let's compare the patterns of cross-correlation for each variable, marking whether each value is significant.  Here we use a filled dot to indicate values significantly different from 0, and a cross to indicate values not significantly different from 0.  We will produce Figure 2 of the Ultee et al manuscript.

In [ ]:
div_colors = 'RdBu' # choose divergent colormap for xcorr
corrnorm_min, corrnorm_max = -0.3, 0.3
lag_colors = 'Greens'
lagnorm_min, lagnorm_max = 0, 365

sig_markers = ['o', 'x']

ls = LightSource(azdeg=225, altdeg=80)

In [ ]:
## set matplotlib font size defaults
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [ ]:
## black-white hillshade topo underneath
rgb2 = ls.shade(np.asarray(b_hel), cmap=plt.get_cmap('gray'), blend_mode='overlay',
               dx=np.mean(np.diff(x_hel)), dy=np.mean(np.diff(y_hel)), vert_exag=5.)

fig, ((ax1, ax2, ax3), (ax4,ax5,ax6)) = plt.subplots(nrows=2,ncols=3, figsize=(12, 8), 
                                                     # constrained_layout=True, 
                                                     sharex=True, sharey=True,
                                                     gridspec_kw={'wspace':0.01})
    
ax1.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc1 = ax1.scatter(np.asarray(xys)[smb_significance,0], np.asarray(xys)[smb_significance,1], 
                  c=np.asarray(smb_corr_amax)[smb_significance], cmap=div_colors, marker=sig_markers[0], 
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax1.scatter(np.asarray(xys)[np.invert(smb_significance),0], np.asarray(xys)[np.invert(smb_significance),1], 
                  c=np.asarray(smb_corr_amax)[np.invert(smb_significance)], cmap=div_colors, marker=sig_markers[1], 
                  vmin=corrnorm_min, vmax=corrnorm_max) #different marker for insig values
ax1.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
        ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
       ylabel='Northing [km]', title='Catchment SMB')

ax2.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc2 = ax2.scatter(np.asarray(xys)[runoff_significance,0], np.asarray(xys)[runoff_significance,1], 
                  c=np.asarray(runoff_corr_amax)[runoff_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax2.scatter(np.asarray(xys)[np.invert(runoff_significance),0], np.asarray(xys)[np.invert(runoff_significance),1],
            c=np.asarray(runoff_corr_amax)[np.invert(runoff_significance)], cmap=div_colors, marker=sig_markers[1],
            vmin=corrnorm_min, vmax=corrnorm_max) # distinguish insig values
ax2.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
       title='Catchment runoff')

ax3.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc3 = ax3.scatter(np.asarray(xys)[terminus_significance,0], np.asarray(xys)[terminus_significance,1], 
                  c=np.asarray(terminus_corr_amax)[terminus_significance], cmap=div_colors, marker=sig_markers[0],
                  vmin=corrnorm_min, vmax=corrnorm_max)
ax3.scatter(np.asarray(xys)[np.invert(terminus_significance),0], np.asarray(xys)[np.invert(terminus_significance),1],
            c=np.asarray(terminus_corr_amax)[np.invert(terminus_significance)], cmap=div_colors, marker=sig_markers[1],
            vmin=corrnorm_min, vmax=corrnorm_max)

div3 = make_axes_locatable(ax3)
cax3 = div3.append_axes("right", size="5%", pad=0.1)
cb3 = fig.colorbar(sc3, cax=cax3)
cb3.ax.set_ylabel('AMax. xcorr')
ax3.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
       title='Terminus position', aspect=1.)

## SECOND ROW
ax4.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc4 = ax4.scatter(np.asarray(xys)[smb_significance,0], np.asarray(xys)[smb_significance,1], 
                  c=np.asarray(smb_lag_amax)[smb_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax4.scatter(np.asarray(xys)[np.invert(smb_significance),0], np.asarray(xys)[np.invert(smb_significance),1], 
            c=np.asarray(smb_lag_amax)[np.invert(smb_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax4.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]', ylabel='Northing [km]')

ax5.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc5 = ax5.scatter(np.asarray(xys)[runoff_significance,0], np.asarray(xys)[runoff_significance,1], 
                  c=np.asarray(runoff_lag_amax)[runoff_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax5.scatter(np.asarray(xys)[np.invert(runoff_significance),0], np.asarray(xys)[np.invert(runoff_significance),1], 
            c=np.asarray(runoff_lag_amax)[np.invert(runoff_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax5.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]')

ax6.imshow(rgb2, origin='lower', extent=(x_hel[0], x_hel[-1], y_hel[0], y_hel[-1]))
sc6 = ax6.scatter(np.asarray(xys)[terminus_significance,0], np.asarray(xys)[terminus_significance,1], 
                  c=np.asarray(terminus_lag_amax)[terminus_significance], cmap=lag_colors, marker=sig_markers[0],
                  vmin=lagnorm_min, vmax=lagnorm_max)
ax6.scatter(np.asarray(xys)[np.invert(terminus_significance),0], np.asarray(xys)[np.invert(terminus_significance),1], 
            c=np.asarray(terminus_lag_amax)[np.invert(terminus_significance)], cmap=lag_colors, marker=sig_markers[1],
                  vmin=lagnorm_min, vmax=lagnorm_max)

div6 = make_axes_locatable(ax6)
cax6 = div6.append_axes("right", size="5%", pad=0.1)
cb6 = fig.colorbar(sc6, cax=cax6)
cb6.ax.set_ylabel('Lag [d] at peak xcorr')
cb6.set_ticks([0, 60, 120, 180, 240, 300, 360])
ax6.set(xlim=(278000, 320000), xticks=(280000, 300000, 320000), 
      ylim=(-2590000, -2550000), yticks=(-2590000, -2570000, -2550000), 
       xticklabels=('280', '300', '320'), yticklabels=('-2590', '-2570', '-2550'),
      xlabel='Easting [km]', aspect=1.)
# plt.tight_layout()
# plt.show()
# plt.savefig('/Users/lizz/Desktop/20210204-helheim-xcorr_lag_composite')

## Annual chunks to compare changing seasonal cycle

We break signals into annual subsets and compute the cross-correlation signal for each single year of data.

In [ ]:
rf_annual_corrs = []
rf_annual_lags = []
rf_annual_ci = []

point_to_plot =5
date_chks = range(2009, 2017)
for i in range(len(date_chks)-1):
#     snippet = rf[rf[:,0]>=date_chks[i]]
#     snpt = snippet[snippet[:,0]<date_chks[i+1]]
#     d_chk = [d for d in d_interp if (d>=date_chks[i] and d<=date_chks[i+1])]
    corr, lags, ci = nifl.Xcorr1D(xys[point_to_plot], series_func=runoff_func, series_dates=d_interp, 
                              velocity_pred=preds[point_to_plot], t_grid=t_grid, t_limits=(date_chks[i], date_chks[i+1]),
                                  diff=1, normalize=True)
    rf_annual_corrs.append(corr)
    rf_annual_lags.append(lags)
    rf_annual_ci.append(ci)

In [ ]:
# fig, axs = plt.subplots(len(rf_annual_corrs))
# for j in range(len(rf_annual_corrs)):
#     axs[j].plot(rf_annual_lags[j], rf_annual_corrs[j])
#     axs[j].plot(rf_annual_lags[j], rf_annual_ci[j], ls=':', color='k')
#     axs[j].plot(rf_annual_lags[j], -1*np.array(rf_annual_ci[j]), ls=':', color='k')

for j in range(len(rf_annual_corrs)):
    fig, ax = plt.subplots(1)
    ax.axvline(x=0, ls='-', color='k', alpha=0.5)
    ax.axhline(y=0, ls='-', color='k', alpha=0.5)
    ax.plot(rf_annual_lags[j], rf_annual_corrs[j])
    ax.plot(rf_annual_lags[j], rf_annual_ci[j], ls=':', color='k')
    ax.plot(rf_annual_lags[j], -1*np.array(rf_annual_ci[j]), ls=':', color='k')
    ax.set(ylim=(-1,1), title='Xcorr runoff-vel, {}'.format(date_chks[j]), 
           xlabel='Lag [days]', ylabel='xcorr')

Let's compare with the overall signal from the full period.

In [ ]:
corr, lags, ci = nifl.Xcorr1D(xys[5], series_func=runoff_func, series_dates=d_interp, 
                          velocity_pred=preds[5], t_grid=t_grid, t_limits=(2009,2017), diff=1, normalize=True)
fig, ax = plt.subplots(1)
ax.axvline(x=0, ls='-', color='k', alpha=0.5)
ax.axhline(y=0, ls='-', color='k', alpha=0.5)
ax.plot(lags, corr)
ax.plot(lags, ci, ls=':', color='k')
ax.plot(lags, -1*np.array(ci), ls=':', color='k')
ax.set(ylim=(-1,1), title='Xcorr runoff-vel, 2009-2017', xlabel='Lag [days]', ylabel='xcorr');